In [3]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd
import joblib

# Read the CSV and Perform Basic Data Cleaning

In [5]:
mbti_df = pd.read_csv("Resources/mbti.csv")
mbti_df.head()

,type,posts,description,words_per_comment,squared_total_words,word_count_variance_per_comment,interrobangs_per_comment,i_e,n_s,f_t,j_p
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,Introvert Intuition Feeling Judging,11.12,22.24,135.2900,0.42,I,N,F,J
1,ENTP,'I'm finding the lack of me in these posts ver...,Extrovert Intuition Thinking Perceiving,23.40,46.80,187.4756,0.10,E,N,T,P
2,INTP,'Good one _____ https://www.youtube.com/wat...,Introvert Intuition Thinking Perceiving,16.72,33.44,180.6900,0.32,I,N,T,P
3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",Introvert Intuition Thinking Judging,21.28,42.56,181.8324,0.28,I,N,T,J
4,ENTJ,'You're fired.|||That's another silly misconce...,Extrovert Intuition Thinking Judging,19.34,38.68,196.4576,0.22,E,N,T,J


# Select features (columns)

In [6]:
mbti_df.columns

Index(['type', 'posts', 'description', 'words_per_comment',
       'squared_total_words', 'word_count_variance_per_comment',
       'interrobangs_per_comment', 'i_e', 'n_s', 'f_t', 'j_p'],
      dtype='object')

In [7]:
# Set features. This will also be used as your x values.
selected_features = mbti_df[['words_per_comment','squared_total_words','squared_total_words','interrobangs_per_comment']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
target = mbti_df["type"]
target_names = ["INFJ","INFP","INTJ",'INTP',"ISFJ","ISFP","ISTJ",'ISTP',"ENFJ","ENFP","ENTJ",'ENTP',"ESFJ","ESFP","ESTJ",'ESTP']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

In [10]:
X_train.head()

,words_per_comment,squared_total_words,squared_total_words,interrobangs_per_comment
2706,28.22,56.44,56.44,1.14
2521,20.92,41.84,41.84,0.24
4192,25.90,51.80,51.80,0.74
6296,30.04,60.08,60.08,0.16
3399,28.98,57.96,57.96,0.60


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [12]:
# Scale your data
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [14]:
from tensorflow.keras.utils import to_categorical

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [15]:
X_train_scaled.shape[1]

4

In [16]:
y_train_categorical.shape[1]

16

# Train the Model



In [17]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=16, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

Using TensorFlow backend.


Epoch 1/100
 - 1s - loss: 2.3674 - accuracy: 0.1931
Epoch 2/100
 - 1s - loss: 2.2665 - accuracy: 0.2043
Epoch 3/100
 - 1s - loss: 2.2553 - accuracy: 0.2052
Epoch 4/100
 - 1s - loss: 2.2511 - accuracy: 0.2086
Epoch 5/100
 - 1s - loss: 2.2497 - accuracy: 0.2086
Epoch 6/100
 - 1s - loss: 2.2476 - accuracy: 0.2097
Epoch 7/100
 - 1s - loss: 2.2471 - accuracy: 0.2092
Epoch 8/100
 - 1s - loss: 2.2442 - accuracy: 0.2106
Epoch 9/100
 - 1s - loss: 2.2447 - accuracy: 0.2084
Epoch 10/100
 - 1s - loss: 2.2442 - accuracy: 0.2127
Epoch 11/100
 - 1s - loss: 2.2437 - accuracy: 0.2175
Epoch 12/100
 - 1s - loss: 2.2426 - accuracy: 0.2106
Epoch 13/100
 - 1s - loss: 2.2419 - accuracy: 0.2067
Epoch 14/100
 - 1s - loss: 2.2412 - accuracy: 0.2126
Epoch 15/100
 - 1s - loss: 2.2402 - accuracy: 0.2117
Epoch 16/100
 - 1s - loss: 2.2402 - accuracy: 0.2115
Epoch 17/100
 - 1s - loss: 2.2396 - accuracy: 0.2121
Epoch 18/100
 - 1s - loss: 2.2378 - accuracy: 0.2140
Epoch 19/100
 - 1s - loss: 2.2384 - accuracy: 0.2098
Ep

In [18]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 2.2551888107099307, Accuracy: 0.21807284653186798


# Re-test for I vs. E

In [19]:
target = mbti_df["i_e"]
target_names = ["Introvert","Extrovert"]

X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)
X_train.head()

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"I vs. E - Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 - 1s - loss: 0.5554 - accuracy: 0.7613
Epoch 2/100
 - 1s - loss: 0.5374 - accuracy: 0.7650
Epoch 3/100
 - 1s - loss: 0.5373 - accuracy: 0.7658
Epoch 4/100
 - 1s - loss: 0.5370 - accuracy: 0.7654
Epoch 5/100
 - 1s - loss: 0.5367 - accuracy: 0.7650
Epoch 6/100
 - 1s - loss: 0.5357 - accuracy: 0.7664
Epoch 7/100
 - 1s - loss: 0.5362 - accuracy: 0.7661
Epoch 8/100
 - 1s - loss: 0.5361 - accuracy: 0.7661
Epoch 9/100
 - 1s - loss: 0.5358 - accuracy: 0.7641
Epoch 10/100
 - 1s - loss: 0.5351 - accuracy: 0.7661
Epoch 11/100
 - 1s - loss: 0.5355 - accuracy: 0.7653
Epoch 12/100
 - 1s - loss: 0.5350 - accuracy: 0.7665
Epoch 13/100
 - 1s - loss: 0.5350 - accuracy: 0.7658
Epoch 14/100
 - 1s - loss: 0.5350 - accuracy: 0.7661
Epoch 15/100
 - 1s - loss: 0.5356 - accuracy: 0.7659
Epoch 16/100
 - 1s - loss: 0.5352 - accuracy: 0.7662
Epoch 17/100
 - 1s - loss: 0.5345 - accuracy: 0.7667
Epoch 18/100
 - 1s - loss: 0.5344 - accuracy: 0.7650
Epoch 19/100
 - 1s - loss: 0.5340 - accuracy: 0.7664
Ep

# Re-test for N vs. S

In [20]:
target = mbti_df["n_s"]
target_names = ["Intuition","Sensing"]

X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)
X_train.head()

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"N vs. S - Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 - 2s - loss: 0.4350 - accuracy: 0.8514
Epoch 2/100
 - 1s - loss: 0.4027 - accuracy: 0.8632
Epoch 3/100
 - 1s - loss: 0.4010 - accuracy: 0.8632
Epoch 4/100
 - 1s - loss: 0.4011 - accuracy: 0.8632
Epoch 5/100
 - 1s - loss: 0.3993 - accuracy: 0.8632
Epoch 6/100
 - 1s - loss: 0.3994 - accuracy: 0.8632
Epoch 7/100
 - 1s - loss: 0.3990 - accuracy: 0.8632
Epoch 8/100
 - 1s - loss: 0.3996 - accuracy: 0.8632
Epoch 9/100
 - 1s - loss: 0.3985 - accuracy: 0.8632
Epoch 10/100
 - 1s - loss: 0.3993 - accuracy: 0.8632
Epoch 11/100
 - 1s - loss: 0.3980 - accuracy: 0.8632
Epoch 12/100
 - 1s - loss: 0.3984 - accuracy: 0.8632
Epoch 13/100
 - 1s - loss: 0.3986 - accuracy: 0.8632
Epoch 14/100
 - 1s - loss: 0.3987 - accuracy: 0.8632
Epoch 15/100
 - 1s - loss: 0.3981 - accuracy: 0.8632
Epoch 16/100
 - 1s - loss: 0.3986 - accuracy: 0.8632
Epoch 17/100
 - 1s - loss: 0.3988 - accuracy: 0.8632
Epoch 18/100
 - 1s - loss: 0.3986 - accuracy: 0.8632
Epoch 19/100
 - 1s - loss: 0.3982 - accuracy: 0.8632
Ep

# Re-test for F vs. J

In [21]:
target = mbti_df["f_t"]
target_names = ["Feeling","Thinking"]

X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)
X_train.head()

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"F vs. J - Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 - 1s - loss: 0.6761 - accuracy: 0.5653
Epoch 2/100
 - 0s - loss: 0.6728 - accuracy: 0.5765
Epoch 3/100
 - 0s - loss: 0.6730 - accuracy: 0.5675
Epoch 4/100
 - 0s - loss: 0.6727 - accuracy: 0.5749
Epoch 5/100
 - 0s - loss: 0.6712 - accuracy: 0.5733
Epoch 6/100
 - 1s - loss: 0.6714 - accuracy: 0.5729
Epoch 7/100
 - 1s - loss: 0.6714 - accuracy: 0.5741
Epoch 8/100
 - 1s - loss: 0.6701 - accuracy: 0.5759
Epoch 9/100
 - 1s - loss: 0.6696 - accuracy: 0.5741
Epoch 10/100
 - 1s - loss: 0.6703 - accuracy: 0.5798
Epoch 11/100
 - 1s - loss: 0.6702 - accuracy: 0.5750
Epoch 12/100
 - 1s - loss: 0.6697 - accuracy: 0.5778
Epoch 13/100
 - 1s - loss: 0.6695 - accuracy: 0.5769
Epoch 14/100
 - 1s - loss: 0.6699 - accuracy: 0.5719
Epoch 15/100
 - 1s - loss: 0.6687 - accuracy: 0.5787
Epoch 16/100
 - 1s - loss: 0.6695 - accuracy: 0.5749
Epoch 17/100
 - 1s - loss: 0.6696 - accuracy: 0.5802
Epoch 18/100
 - 1s - loss: 0.6694 - accuracy: 0.5773
Epoch 19/100
 - 1s - loss: 0.6688 - accuracy: 0.5765
Ep

# Re-test for J vs. P

In [22]:
target = mbti_df["j_p"]
target_names = ["Judging","Perceiving"]

X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)
X_train.head()

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=4))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"J vs. P - Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 - 2s - loss: 0.6773 - accuracy: 0.5933
Epoch 2/100
 - 1s - loss: 0.6736 - accuracy: 0.5990
Epoch 3/100
 - 1s - loss: 0.6721 - accuracy: 0.6033
Epoch 4/100
 - 2s - loss: 0.6716 - accuracy: 0.5988
Epoch 5/100
 - 1s - loss: 0.6718 - accuracy: 0.5970
Epoch 6/100
 - 1s - loss: 0.6724 - accuracy: 0.6004
Epoch 7/100
 - 2s - loss: 0.6715 - accuracy: 0.5982
Epoch 8/100
 - 2s - loss: 0.6713 - accuracy: 0.6019
Epoch 9/100
 - 2s - loss: 0.6707 - accuracy: 0.6010
Epoch 10/100
 - 2s - loss: 0.6707 - accuracy: 0.6001
Epoch 11/100
 - 2s - loss: 0.6707 - accuracy: 0.6002
Epoch 12/100
 - 2s - loss: 0.6709 - accuracy: 0.6013
Epoch 13/100
 - 2s - loss: 0.6700 - accuracy: 0.6041
Epoch 14/100
 - 2s - loss: 0.6705 - accuracy: 0.6013
Epoch 15/100
 - 2s - loss: 0.6702 - accuracy: 0.6016
Epoch 16/100
 - 1s - loss: 0.6703 - accuracy: 0.6014
Epoch 17/100
 - 1s - loss: 0.6701 - accuracy: 0.6002
Epoch 18/100
 - 2s - loss: 0.6704 - accuracy: 0.6013
Epoch 19/100
 - 1s - loss: 0.6702 - accuracy: 0.6007
Ep

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [32]:
# Tune Batch Size and Number of Epochs
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
def create_model(init_mode='uniform'):
    # define model
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=17))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=3, activation='softmax'))
    # compile model
    model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )
    return model
seed = 7
np.random.seed(seed)
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
model_CV = KerasClassifier(build_fn=create_model, epochs=epochs, 
                           batch_size=batch_size, verbose=2)
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(model_CV, param_grid, n_jobs=-1, cv=3)

In [34]:
grid_result=grid1.fit(X_train_scaled, y_train_categorical)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
 - 1s - loss: 0.7456 - accuracy: 0.6468
Epoch 2/50
 - 1s - loss: 0.6633 - accuracy: 0.7019
Epoch 3/50
 - 1s - loss: 0.6343 - accuracy: 0.7130
Epoch 4/50
 - 1s - loss: 0.6172 - accuracy: 0.7187
Epoch 5/50
 - 1s - loss: 0.6046 - accuracy: 0.7271
Epoch 6/50
 - 1s - loss: 0.5880 - accuracy: 0.7305
Epoch 7/50
 - 1s - loss: 0.5813 - accuracy: 0.7406
Epoch 8/50
 - 1s - loss: 0.5756 - accuracy: 0.7391
Epoch 9/50
 - 1s - loss: 0.5654 - accuracy: 0.7471
Epoch 10/50
 - 1s - loss: 0.5607 - accuracy: 0.7477
Epoch 11/50
 - 1s - loss: 0.5573 - accuracy: 0.7465
Epoch 12/50
 - 1s - loss: 0.5505 - accuracy: 0.7465
Epoch 13/50
 - 1s - loss: 0.5412 - accuracy: 0.7475
Epoch 14/50
 - 1s - loss: 0.5381 - accuracy: 0.7580
Epoch 15/50
 - 1s - loss: 0.5306 - accuracy: 0.7572
Epoch 16/50
 - 1s - loss: 0.5264 - accuracy: 0.7644
Epoch 17/50
 - 1s - loss: 0.5259 - accuracy: 0.7656
Epoch 18/50
 - 1s - loss: 0.5150 - accuracy: 0.7660
Epoch 19/50
 - 1s - loss: 0.5115 - accuracy: 0.7704
Epoch 20/50
 - 1s - l

In [39]:
# Tune the Training Optimization Algorithm
def create_model1(optimizer='adam'):
    # define model
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=17))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=3, activation='softmax'))
    # compile model
    model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )
    return model
model_CV1 = KerasClassifier(build_fn=create_model1, epochs=50, batch_size=20, verbose=2)
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid1 = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model_CV1, param_grid=param_grid1, n_jobs=-1, cv=3)
grid_result1 = grid.fit(X_train_scaled, y_train_categorical)

print("Best: %f using %s" % (grid_result1.best_score_, grid_result1.best_params_))
means = grid_result1.cv_results_['mean_test_score']
stds = grid_result1.cv_results_['std_test_score']
params = grid_result1.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
 - 1s - loss: 0.7094 - accuracy: 0.6769
Epoch 2/50
 - 1s - loss: 0.6565 - accuracy: 0.7038
Epoch 3/50
 - 1s - loss: 0.6383 - accuracy: 0.7149
Epoch 4/50
 - 1s - loss: 0.6261 - accuracy: 0.7236
Epoch 5/50
 - 1s - loss: 0.6183 - accuracy: 0.7219
Epoch 6/50
 - 1s - loss: 0.6102 - accuracy: 0.7280
Epoch 7/50
 - 1s - loss: 0.6048 - accuracy: 0.7349
Epoch 8/50
 - 1s - loss: 0.5988 - accuracy: 0.7345
Epoch 9/50
 - 1s - loss: 0.5947 - accuracy: 0.7364
Epoch 10/50
 - 1s - loss: 0.5904 - accuracy: 0.7393
Epoch 11/50
 - 1s - loss: 0.5873 - accuracy: 0.7400
Epoch 12/50
 - 1s - loss: 0.5833 - accuracy: 0.7376
Epoch 13/50
 - 1s - loss: 0.5802 - accuracy: 0.7406
Epoch 14/50
 - 1s - loss: 0.5776 - accuracy: 0.7402
Epoch 15/50
 - 1s - loss: 0.5752 - accuracy: 0.7421
Epoch 16/50
 - 1s - loss: 0.5720 - accuracy: 0.7459
Epoch 17/50
 - 1s - loss: 0.5701 - accuracy: 0.7454
Epoch 18/50
 - 1s - loss: 0.5686 - accuracy: 0.7461
Epoch 19/50
 - 1s - loss: 0.5654 - accuracy: 0.7479
Epoch 20/50
 - 1s - l

In [40]:
# Tune the Neuron Activation Function
def create_model2(activation='relu'):
    # define model
    model = Sequential()
    model.add(Dense(units=100, activation=activation, input_dim=17))
    model.add(Dense(units=100, activation=activation))
    model.add(Dense(units=3, activation='softmax'))
    # compile model
    model.compile(
    optimizer='Adagrad',
    loss='categorical_crossentropy',
    metrics=['accuracy']
    )
    return model
model_CV2 = KerasClassifier(build_fn=create_model2, epochs=50, batch_size=20, verbose=2)
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid2 = dict(activation=activation)
grid = GridSearchCV(estimator=model_CV2, param_grid=param_grid2, n_jobs=-1, cv=3)

grid_result2 = grid.fit(X_train_scaled, y_train_categorical)

print("Best: %f using %s" % (grid_result2.best_score_, grid_result2.best_params_))
means = grid_result2.cv_results_['mean_test_score']
stds = grid_result2.cv_results_['std_test_score']
params = grid_result2.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/50
 - 1s - loss: 0.7080 - accuracy: 0.6790
Epoch 2/50
 - 1s - loss: 0.6498 - accuracy: 0.7118
Epoch 3/50
 - 1s - loss: 0.6316 - accuracy: 0.7173
Epoch 4/50
 - 1s - loss: 0.6200 - accuracy: 0.7263
Epoch 5/50
 - 1s - loss: 0.6113 - accuracy: 0.7301
Epoch 6/50
 - 1s - loss: 0.6033 - accuracy: 0.7330
Epoch 7/50
 - 1s - loss: 0.5977 - accuracy: 0.7358
Epoch 8/50
 - 1s - loss: 0.5926 - accuracy: 0.7345
Epoch 9/50
 - 1s - loss: 0.5872 - accuracy: 0.7383
Epoch 10/50
 - 1s - loss: 0.5842 - accuracy: 0.7385
Epoch 11/50
 - 1s - loss: 0.5804 - accuracy: 0.7461
Epoch 12/50
 - 1s - loss: 0.5775 - accuracy: 0.7419
Epoch 13/50
 - 1s - loss: 0.5736 - accuracy: 0.7423
Epoch 14/50
 - 1s - loss: 0.5713 - accuracy: 0.7429
Epoch 15/50
 - 1s - loss: 0.5690 - accuracy: 0.7482
Epoch 16/50
 - 1s - loss: 0.5661 - accuracy: 0.7479
Epoch 17/50
 - 1s - loss: 0.5636 - accuracy: 0.7467
Epoch 18/50
 - 1s - loss: 0.5616 - accuracy: 0.7480
Epoch 19/50
 - 1s - loss: 0.5602 - accuracy: 0.7492
Epoch 20/50
 - 1s - l

In [64]:
predictions = grid.predict(X_test_scaled)
predictions=to_categorical(predictions)

from sklearn.metrics import classification_report
print(classification_report(y_test_categorical, predictions, target_names=target_names))

                precision    recall  f1-score   support

     CONFIRMED       0.48      0.42      0.45       411
FALSE POSITIVE       0.67      0.72      0.69       484
     CANDIDATE       0.84      0.85      0.85       853

     micro avg       0.71      0.71      0.71      1748
     macro avg       0.66      0.66      0.66      1748
  weighted avg       0.71      0.71      0.71      1748
   samples avg       0.71      0.71      0.71      1748



# Save the Model

In [71]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
filename = 'Neural.sav'
joblib.dump(model, 'Neural.sav')

['Neural.sav']